In [2]:
# Import Numpy
import numpy as np
# For timer
import time
# Import torch
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
# Summary-like from Tensorflow
from torchsummary import summary
# Import plotter
import matplotlib.pyplot as plt

# Import local libraries
import sys
sys.path.insert(0, '../src/ecg_denoiser/ecg_toolkit')
sys.path.insert(0, '../src/ecg_denoiser/local_models')
sys.path.insert(0, '../data/')
# ==== LOCAL LIBS ====
import ecg_tools_lite as etl
import trainer as tr
import models
# import ecg_tools as et

In [3]:
# Set date/time for logging:
from datetime import datetime
today = datetime.today().strftime("%B %d, %Y || %H:%M:%S")
def print_today():
    print("Ran on: ", today)

In [ ]:
# Set model parameters here:
epochs = 10
norm_option = 2

In [ ]:
ecg_noisy = torch.from_numpy(np.load('../data/all_none_fs1024_e06.npy')).cuda().double()
ecg_clean = torch.from_numpy(np.load('../data/all_none_fs1024_e24.npy')).cuda().double()

# Acquire original shape
x = ecg_noisy.shape[0]
y = ecg_noisy.shape[2]
z = ecg_noisy.shape[1]
print("Original shape was: ", x, z, y )

# Reshape
ecg_noisy = torch.reshape( ecg_noisy, (x, y, z))
ecg_clean = torch.reshape( ecg_clean, (x, y, z))

print(f'New shape: {ecg_noisy.shape}')

# == Normalization technique (Norm from -1 to 1)
ecg_noisy = etl.norm_sig( ecg_noisy )
ecg_clean = etl.norm_sig( ecg_clean )
print("Data has been normalized with range -1 to 1")


## Start building the Model

In [ ]:
# ----------- Prepare the model ----------- 
# set model save name based on date
model_save_name = 'model_' + str(etl.get_local_time()) + '.pt'
print("The resulting model name when saved is: ", model_save_name)
cnn_model = models.cnn_denoiser().cuda()
#  Show summary for the model
summary( cnn_model, (1, 1024)) # can only work if sent to cuda
# Convert to double
cnn_model.double()


### Create a new Model

In [ ]:
# Start training
# Estimate run time of around 30 mins
losses = tr.train_model( model=cnn_model,
                    epochs=epochs, 
                    ecg_noisy=ecg_noisy, 
                    ecg_clean=ecg_clean,
                    train_pct=0.8)
# Save the model into dictionary
torch.save(  cnn_model.state_dict(), model_save_name)
print("Saved on ",  today)
# -------------------------------
print_today()

### Load an existing model

In [ ]:
# cnn_model.load_state_dict( torch.load("E:\Developer\ecg_thesis_code\experiments\model_2022-11-14_2229.pt") )

# or

cnn_model = torch.load("E:\Developer\ecg_thesis_code\experiments\laurenz_model.pt")

In [ ]:
denoised_result = cnn_model.denoiser(ecg_noisy[0:200])
denoised_result_flat = denoised_result.flatten()

In [ ]:
torch.save(cnn_model, "laurenz_model.pt")

In [ ]:
etl.ecg_plot([ecg_clean.flatten().cpu(), ecg_noisy.flatten().cpu(), denoised_result_flat.detach().cpu()], ['ECG Clean', 'ECG Noisy', 'ECG Result'])

In [ ]:
import py_compile
py_compile.compile("E:\Developer\ecg_thesis_code\local_models\models.py", "models.pyc")